------------------------------------------------------------------------------------------

IMPORTS

---------------------------------------------------------------------

In [2]:
# imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle
from openpyxl.utils import get_column_letter
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import requests
import openpyxl

------------------------------------------------------------------------------

BING WEBSITE SCRAPE

---------------------------------------------------------------------------

In [ ]:
aktueller_index = 0  # Variable zur Verfolgung des aktuellen Index im DataFrame
excel_datei = 'Gewünschte Datei.xlsx'  # Pfad zur Excel-Datei
df = pd.read_excel(excel_datei)  # Einlesen der Excel-Datei in einen DataFrame
spalte_hersteller_ccp = 'Hersteller_CCP'  # Name der Spalte, die den Hersteller enthält

driver = webdriver.Chrome()  # Initialisierung des Chrome Webdrivers
driver.get("https://www.bing.com/search?q=hahaha")  # Öffnen der Bing-Suchseite im Webbrowser

try:
    button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, "bnp_btn_accept")))
    button.click()  # Klick auf den Zustimmungs-Button für Bing-Cookies (sofern vorhanden)
except:
    pass

neue_excel_datei = 'Gewünschter Name (nach ergänzung der Spalte mit Infos).xlsx'  # Name der neuen Excel-Datei
wb = load_workbook(excel_datei)  # Laden der vorhandenen Excel-Datei
wb.save(neue_excel_datei)  # Speichern der Excel-Datei unter neuem Namen

arbeitsmappe = openpyxl.load_workbook(neue_excel_datei)  # Öffnen der neuen Excel-Datei
arbeitsblatt = arbeitsmappe['Sheet1']  # Auswahl des Arbeitsblatts
arbeitsblatt.cell(row=1, column=arbeitsblatt.max_column + 1).value = 'Website'  # Hinzufügen einer Überschrift zur neuen Spalte
arbeitsmappe.save(neue_excel_datei)  # Speichern der aktualisierten Excel-Datei

neuer_df = pd.read_excel(neue_excel_datei)  # Einlesen der neuen Excel-Datei in einen DataFrame

while aktueller_index < len(df):  # Schleife für jeden Eintrag im DataFrame
    wert = df.at[aktueller_index, spalte_hersteller_ccp]  # Wert in der aktuellen Zeile und Spalte abrufen
    if pd.isnull(wert):  # Überprüfen, ob der Wert leer ist
        break  # Schleife beenden, wenn der Wert leer ist

    url = f"https://www.bing.com/search?q={wert}"  # URL für die Bing-Suche mit dem aktuellen Wert

    try:
        driver.get(url)  # Öffnen der Bing-Suche im Webbrowser
        wait = WebDriverWait(driver, 2.5)  # Warten auf das Laden der Seite (möglicherweise zu kurz)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'l_ecrd_webicon_txt')))  # Warten auf das Laden der Suchergebnisse
        W1 = driver.find_element(By.CLASS_NAME, 'l_ecrd_webicon_txt')  # Suchergebnis-Element für die Website
        W2 = W1.get_attribute('textContent')  # Textinhalt des Suchergebnisses

        if W2 == 'Wikipedia':
            pass
        else:
            arbeitsblatt.cell(row=aktueller_index+2, column=arbeitsblatt.max_column).value = W2  # Aktualisieren des Werts in der neuen Excel-Datei
    except:
        pass

    aktueller_index += 1  # Inkrementieren des Index für die nächste Zeile im DataFrame

arbeitsmappe.save(neue_excel_datei)  # Speichern der aktualisierten Excel-Datei
neuer_df = pd.read_excel(neue_excel_datei)  # Einlesen der aktualisierten Excel-Datei in einen DataFrame
driver.quit()  # Beenden des Chrome Webdrivers

---------------------------------------------------------------------------

WEBSITE HYPERLINK

--------------------------------------------------------------------------

In [ ]:
excel_file_path = 'Gewünschter Name (nach ergänzung der Spalte mit Infos).xlsx'  # Pfad zur Excel-Datei
sheet_name = 'Sheet1'  # Name des Arbeitsblatts
column_name = 'Website'  # Name der Spalte, die die Website enthält

workbook = load_workbook(excel_file_path)  # Laden der Excel-Datei
worksheet = workbook[sheet_name]  # Auswahl des Arbeitsblatts
column_index = None

# Durchsuchen der Spalten, um den Index der gewünschten Spalte zu finden
for col in worksheet.iter_cols(1, worksheet.max_column):
    if col[0].value == column_name:
        column_index = col[0].column
        break

if column_index:
    hyperlink_style = "Hyperlink"  # Verwendung des integrierten Hyperlink-Stils

    # Iteration über die Zeilen der Spalte
    for row in worksheet.iter_rows(min_row=2, min_col=column_index, max_col=column_index):
        cell = row[0]

        if cell.value:
            search_query = quote(cell.value)  # Codierung des Suchbegriffs für die URL
            search_link = f'https://{search_query}'  # Erstellen des Suchlinks
            worksheet[cell.coordinate].value = cell.value  # Setzen des Zellenwerts
            worksheet[cell.coordinate].hyperlink = search_link  # Setzen des Hyperlinks
            worksheet[cell.coordinate].style = hyperlink_style  # Setzen des Hyperlink-Stils

workbook.save(excel_file_path)  # Speichern der aktualisierten Excel-Datei